In [1]:
import os
import cv2
import math
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from keras.utils import np_utils
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers.pooling import AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input


from utils import video_to_frame, create_row_pixels

%matplotlib inline

Using TensorFlow backend.


In [2]:
# open the .txt file which have names of training videos
train = pd.read_csv(r"C:\Users\dbernsohn\Documents\projects\SVW\SVW.csv")

train = train[:-1]
train.head()

,FileName,Genre,W,H,# frames,FPS,Length (s),Unnamed: 7,BB available?,Portrait?,Unnamed: 10,Train 1?,Test 1?,Unnamed: 13,Train 2?,Test 2?,Unnamed: 16,Train 3?,Test 3?,Unnamed: 19
0,10157___9040036fdf2c4e8296c38232cf30c6f4.mp4,archery,480.0,270.0,221.0,30.0,7.37,NaN,1.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN
1,10663___26dbed1f45c54e7b972f13fcf91db418.mp4,archery,480.0,270.0,293.0,30.0,9.77,NaN,0.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,0.0,1.0,NaN
2,10821___d8b0d26588cf4d4590e04890154b9325.mp4,archery,480.0,270.0,711.0,30.0,23.70,NaN,0.0,0.0,NaN,0.0,1.0,NaN,1.0,0.0,NaN,0.0,1.0,NaN
3,1114___449ccc63f10f489a9467916fbb768403.mp4,archery,480.0,270.0,421.0,30.0,14.03,NaN,0.0,0.0,NaN,1.0,0.0,NaN,0.0,1.0,NaN,1.0,0.0,NaN
4,11151___f78e6a60418f4e29ad7b3ae9be1a55be.mp4,archery,480.0,270.0,358.0,30.0,11.93,NaN,0.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,0.0,1.0,NaN


In [4]:
try:
    frame_df = pd.read_csv('frame_df.csv')
except:
#     storing the frames from training videos
    frame_df = video_to_frame(train, path_to_videos = r"C:\Users\dbernsohn\Documents\projects\SVW\Videos", path_to_save = r"C:\Users\dbernsohn\Documents\Python Scripts\video_classification\train")
    frame_df.to_csv('frame_df.csv')

#### Build the labels 

In [10]:
print("Train on classes: ", ', '.join(frame_df.Class.unique()[:5]))
data, labels = create_row_pixels(frame_df[frame_df.Class.isin(frame_df.Class.unique()[:5])], 'train')

Train on classes:  archery, baseball, basketball, bmx, bowling


100%|██████████████████████████████████████████████████████████████████████████████| 8412/8412 [01:29<00:00, 94.28it/s]


# of images:  8412


#### Split the data into train/test 

In [11]:
data = np.array(data)
labels = np.array(labels)
 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
 
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.25, stratify=labels, random_state=42)

#### Create Generator 

In [12]:
trainAug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

valAug = ImageDataGenerator()

#### Create model 

In [13]:
baseModel = ResNet50(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

W0928 19:09:05.311324 64744 deprecation_wrapper.py:119] From C:\Users\dbernsohn\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0928 19:09:05.359320 64744 deprecation_wrapper.py:119] From C:\Users\dbernsohn\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0928 19:09:05.378698 64744 deprecation_wrapper.py:119] From C:\Users\dbernsohn\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0928 19:09:05.434737 64744 deprecation_wrapper.py:119] From C:\Users\dbernsohn\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backen

#### Train the model from the last layer

In [14]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / 4)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
    
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
batch_size = 4
H = model.fit_generator(
    trainAug.flow(trainX, trainY, batch_size=batch_size),
    steps_per_epoch=len(trainX) // batch_size,
    validation_data=valAug.flow(testX, testY),
    validation_steps=len(testX) // batch_size,
    callbacks=callbacks_list,
    epochs=10, verbose = 2)

[INFO] compiling model...


W0928 19:09:42.520693 64744 deprecation_wrapper.py:119] From C:\Users\dbernsohn\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



[INFO] training head...


W0928 19:09:46.681573 64744 deprecation.py:323] From C:\Users\dbernsohn\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10


KeyboardInterrupt: 